In [1]:
import pandas as pd
import ipywidgets as pw
from IPython.display import display
import copy
import random
import scipy.stats as st
import matplotlib.pyplot as plt
import numpy as np
import math

In [2]:
def characteristics (series, middle_count = []):
    character = list()
    if len(middle_count) != 0:
        distribution = list((i) for i in zip(list(set(series)), middle_count)) # дискретний статистичний розподіл
        counts = middle_count
    else:
        counts = [series.count(i) for i in list(set(series))]
        distribution = list((i) for i in zip(list(set(series)), counts)) # дискретний статистичний розподіл
        
    #середнє вибіркове
    average = np.mean(series)
    print("\nСереднє вибіркове : {0}\n".format(average))
    character.append(average)
    
    #мода
    moda = [i[0] for i in distribution if i[1] == max(counts)]
    print("Мода : {0}\n".format(moda))
    character.append(moda)
    
    #медіана
    if len(series) % 2 == 1:
        mediana = series[int(len(series)/2)]
        print("Медіана : {}\n".format(mediana))
    else:
        mediana = list([series[int(len(series)/2 - 1)], series[int(len(series)/2)]])
        print("Медіана : {} or {}\n".format(mediana, (mediana[0]+mediana[1])/2))
    character.append(mediana)
    
    #девіація
    deviation = 0;
    for i in distribution:
        deviation += i[1] * ((i[0] - average)**2)
    print("Девіація : {0}\n".format(deviation))
    character.append(deviation)
    
    #варіанса
    varianza = 0;
    for i in series:
        varianza += (i - average) ** 2
    varianza = varianza / (len(series) - 1)
    print("Варіанса : {0}\n".format(varianza))
    character.append(varianza)
    
    #стандарт
    standart = np.sqrt(varianza)
    print("Стандарт : {0}\n".format(standart))
    character.append(standart)
    
    #варіація
    variation = standart / average 
    print("Варіація : {0}\n".format(variation))
    character.append(variation)
    
    #квантилі
    if (len(series) % 4 == 0):
        Q1 = int(np.percentile(series, 25))
        Q2 = int(np.percentile(series, 50))
        Q3 = int(np.percentile(series, 75))
        print("Квантиль порядку 25 : {0}\n".format(Q1))
        print("Квантиль порядку 50 : {0}\n".format(Q2))
        print("Квантиль порядку 75 : {0}\n".format(Q3))
        latitude = Q3 - Q1
        print("Інтерквантильна широта : {0}\n".format(latitude))
    elif (len(series) % 10 == 0):
        percentile = list()
        for i in range (1,10):
            print("Квантиль порядку {0} : {1}\n".format(i*10, int(np.percentile(series,i*10))))
            percentile.append(int(np.percentile(series,i*10)))
            latitude = percentile[-1] - percentile[0]
        print("Інтердецильна широта : {0}\n".format(latitude))
    
    #моменти
    m0 = 1
    m1 = average
    n1 = 0
    n2 = ((len(series) - 1)/len(series))*varianza
    m2 = 0
    for i in series:
        m2 += i ** 2
    m2 = m2 / len(series) 
    m3 = 0
    for i in series:
        m3 += i ** 3
    m3 = m3 / len(series)
    n3 = m3 - 3 * m2 * m1 + 2 * (m1 ** 3)
    m4 = 0
    for i in series:
        m4 += i ** 4
    m4 = m4 / len(series)
    n4 = m4 - 4 * m3 * m1 + 6 * m2 * (m1 ** 2) - 3 * (m1 ** 4) 

    #асиметрія
    asymmetry = n3 / (n2 ** (3/2))
    print("Асиметрія : {0}\n".format(asymmetry))
    if asymmetry > 0:
        print("Статистичний матеріал скошений вправо")
    if asymmetry < 0:
        print("Статистичний матеріал скошений вліво")
    if asymmetry == 0:
        print("Статистичний матеріал розташований симетрично відносно середини інтервалу [X1, Xn]")
    character.append(asymmetry)    
    
    
    #ексцес
    kurtosis = (n4 / (n2 ** 2)) - 3
    print("\n Ексцес : {0}\n".format(kurtosis))
    if kurtosis > 0:
        print("Статистичний матеріал високовершинний")
    if kurtosis < 0:
        print("Статистичний матеріал низьковершинний")
    if kurtosis == 0:
        print("Статистичний матеріал нормальновершинний")    
    character.append(kurtosis)
    return character

In [11]:
def TIMC (a, b, n, kind):
    if n == "" or a == "" or b == "":
        return 0
    
    if kind == 1:
        sample = np.array([random.randint(int(a), int(b)) for x in range (0, int(n))])
        print("Вибірка : {0}\n".format(sample))
        series = list(sample)
        series.sort() # варіаційний ряд
        
        
        unique_elem = list(set(series))
        freq = [series.count(i) for i in list(set(series))]
        
        print("Варіаційний ряд : {0}".format(series))
        q1 = pd.Series(unique_elem, name = "element")
        q2 = pd.Series(freq, name = "frequency")
        q = pd.concat([q1, q2], axis = 1)
        s = copy.deepcopy(q)
        q["w"] = q["frequency"] / int(n)
        display(q)
        characteristics(series)
        summ = 0
        print ("F(x) = ")
        for i in range(len(unique_elem)):
            if i == 0:
                print("| {0}, якщо x <= {1} ".format(float('{:.3f}'.format(summ)), float('{:.3f}'.format(unique_elem[i]))))
            summ += freq[i]/int(n)
            if i == len(unique_elem)-1:
                print("|{0}, якщо x >= {1}".format(float('{:.3f}'.format(summ)), float('{:.3f}'.format(unique_elem[i]))))
                break
            print("| {0}, якщо {1} < x <= {2}".format(float('{:.3f}'.format(summ)), float('{:.3f}'.format(unique_elem[i])), float('{:.3f}'.format(unique_elem[i+1]))))
        
        dod = 0;
        emp = list([0., 0.])
        for i in list(q["w"]):
            dod += i
            emp.append(dod)
            emp.append(dod)
        a2 = list()
        for i in unique_elem:
            a2.append(i)
            a2.append(i)
        
        plt.plot(a2, emp[1:len(emp)-1])
        plt.show()
        s = s.set_index("element")
        s.plot()
        plt.show()
        s.plot(kind = "bar", width = 0.1)
        
    if kind == 2:
        sample = np.array([float('{:.2f}'.format(random.uniform(float(a), float(b)))) for i in range (0, int(n))])
        series = list(sample)
        series.sort()
        print("Вибірка : {0}\n".format(sample))
        print("Варіаційний ряд : {0}".format(series))
        r = 0
        while (not(int(n) > 2**r and int(n) <= 2**(r + 1))):
            r +=1;
            
        intervals = list()
        step = (np.max(sample) - np.min(sample))/r
        z = np.min(sample)
        for i in range(0,r):
            intervals.append((z, z + step))
            z+= step
        
        counts = list()
        for i in range(r):
            count = 0
            for j in series:
                if intervals[i][0] <= j < intervals[i][1] and i != r - 1:
                    count += 1
                else:
                    if intervals[i][0] <= j <= intervals[i][1]:
                        count += 1
            counts.append(count)
        print(counts)
        s1 = pd.Series(intervals, name = "intervals")
        s2 = pd.Series(counts, name = "frequency")
        s = pd.concat([s1,s2], axis = 1)
        s["middle of each"] = [(i[0] + ((i[1]-i[0])/2)) for i in intervals]
        s["w"] = s["frequency"]/int(n)
        display(s)
        ex1 = characteristics(list(s["middle of each"]), list(s["frequency"]))
        
        q1 = pd.Series(list(set(series)), name = "element")
        q2 = pd.Series([series.count(i) for i in list(set(series))], name = "frequency")
        q = pd.concat([q1, q2], axis = 1)
        
        
        unique_elem = list(s["middle of each"])
        freq = [series.count(i) for i in list(set(series))]
        
        summ = 0
        
        print ("F(x) = ")
        for i in range(len(unique_elem)):
            if i == 0:
                print("| {0}, якщо x <= {1} ".format(float('{:.3f}'.format(summ)), float('{:.3f}'.format(unique_elem[i]))))
            summ += s["w"][i]
            if i == len(unique_elem)-1:
                print("|{0}, якщо x >= {1}".format(float('{:.3f}'.format(1)), float('{:.3f}'.format(unique_elem[i]))))
                break
            print("| {0}, якщо {1} < x <= {2}".format(float('{:.3f}'.format(summ)), float('{:.3f}'.format(unique_elem[i])), float('{:.3f}'.format(unique_elem[i+1]))))

        dod = 0;
        emp = list([0., 0.])
        for i in list(s["w"]):
            dod += i
            emp.append(dod)
            emp.append(dod)
        a2 = list()
        for i in unique_elem:
            a2.append(i)
            a2.append(i)
        
        plt.plot(a2, emp[1:len(emp)-1])
        plt.show()     
        
        plt.bar(list(s["middle of each"]), list(s["frequency"]), width = 0.1)
        plt.show()
        
        display(q)
        ex2 = characteristics(series)
        rng = np.random.RandomState(42)
        compare = pd.DataFrame([ex1,
                                ex2],
                  columns=['середнє', 'мода', 'медіана', 'девіація', 'варіанса', 'стандарт', 'варіація', 'асиметрія', 'ексцес'], index = ['інтервали', 'вибірка'])
        display(compare)
        
        plt.hist(series, bins = r)

In [13]:
pw.interactive(TIMC, a = "", b = "", n = "", kind = {'Дискретна випадкова змінна' : 1, 'Неперервна випадкова змінна' : 2})

interactive(children=(Text(value='', description='a'), Text(value='', description='b'), Text(value='', descrip…